In [1]:
# To download dependencies, execute the following command in the terminal:
# ❯ spark-submit --packages com.databricks:spark-xml_2.12:0.13.0 xml.py

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# Initialize SparkSession
spark = (
    SparkSession.builder
    .appName("ReadXML")
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")
    .getOrCreate()
        )

:: loading settings :: url = jar:file:/Users/victorgalan/miniconda3/envs/delta_env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/victorgalan/.ivy2/cache
The jars for the packages stored in: /Users/victorgalan/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a5b603e-215f-427b-bc5d-7fe6ec162c09;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.13.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 1171ms :: artifacts dl 10ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.13.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf    

In [3]:
# Define the path to your XML file
xml_file_path = "./input/books.xml"

# Define row tag to be extracted
rowTag = "book"

In [4]:
# Read XML file into DataFrame
df = (
    spark.read
    .format('com.databricks.spark.xml')
    .option("rowTag", rowTag)
    .load(xml_file_path)
)

In [5]:
print("Reading XML...")
# Print DataFrame schema
df.printSchema()

Reading XML...
root
 |-- _id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- description: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)
 |-- title: string (nullable = true)



In [6]:
df.show(truncate=False)

+-----+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
(
    df.write
    .format("parquet")
    .mode("overwrite")
    .saveAsTable("xml_tb")
)

In [8]:
(
    df.write
    .format("parquet")
    .mode("overwrite")
    .save("xml_parquet_tb")
)

In [10]:
(
    spark.sql(
        """
        show tables
        """
    )
).show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|   xml_tb|      false|
+---------+---------+-----------+



In [13]:
(
    spark.sql(
        """
        SELECT * 
        FROM default.xml_tb
        limit 5
        """
    )
).show()

+-----+--------------------+--------------------+--------+-----+------------+--------------------+
|  _id|              author|         description|   genre|price|publish_date|               title|
+-----+--------------------+--------------------+--------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|\n\n\n         An...|Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...| Fantasy| 5.95|  2000-12-16|       Midnight Rain|
|bk103|         Corets, Eva|After the collaps...| Fantasy| 5.95|  2000-11-17|     Maeve Ascendant|
|bk104|         Corets, Eva|In post-apocalyps...| Fantasy| 5.95|  2001-03-10|     Oberon's Legacy|
|bk105|         Corets, Eva|The two daughters...| Fantasy| 5.95|  2001-09-10|  The Sundered Grail|
+-----+--------------------+--------------------+--------+-----+------------+--------------------+



In [14]:
(
    spark.read
    .format("parquet")
    .load("xml_parquet_tb")
    .limit(4)
).show()

+-----+--------------------+--------------------+--------+-----+------------+--------------------+
|  _id|              author|         description|   genre|price|publish_date|               title|
+-----+--------------------+--------------------+--------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|\n\n\n         An...|Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...| Fantasy| 5.95|  2000-12-16|       Midnight Rain|
|bk103|         Corets, Eva|After the collaps...| Fantasy| 5.95|  2000-11-17|     Maeve Ascendant|
|bk104|         Corets, Eva|In post-apocalyps...| Fantasy| 5.95|  2001-03-10|     Oberon's Legacy|
+-----+--------------------+--------------------+--------+-----+------------+--------------------+



In [15]:
# Stop SparkSession
spark.stop()